In [82]:
import pandas as pd
import numpy as np
import re
import string
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk import word_tokenize
from string import punctuation
from nltk.corpus import stopwords 
from nltk.util import ngrams
from collections import Counter
df=pd.read_csv('indeed.csv')

In [83]:
df.head()

,web-scraper-order,pros,cons,time,rating,tenure,recommend,jobtitle,advice,title
0,1538929161-1013,1. Simple product (I mean that as a huge compl...,My experience at Indeed truly was a tale of tw...,"Aug 7, 2017","<span class=""rating""><span class=""value-title""...",I worked at Indeed full-time,NaN,Former Employee - Anonymous Employee,"1. Act globally, don't just talk about it\nDur...","""A Tale of Two Managers"""
1,1538929049-740,"Great company and really nice people, i feel l...",none whatsoever the company is great all of my...,"Jul 28, 2016","<span class=""rating""><span class=""value-title""...",I have been working at Indeed (More than a year),Recommends,Current Employee - Social Media Strategist/Mar...,dont have any,"""Account Executive"""
2,1538929155-995,"Career development programs, bonuses based on ...","In becoming a big corporate, Indeed is losing ...","Sep 14, 2017","<span class=""rating""><span class=""value-title""...",I worked at Indeed full-time,Recommends,Former Employee - Anonymous Employee,Improve Communication,"""Client Services"""
3,1538929026-683,"unlimited PTO, great team, free food","favoritism, lack of professionalism and knowle...","Mar 1, 2017","<span class=""rating""><span class=""value-title""...",I have been working at Indeed full-time (More ...,Doesn't Recommend,Current Employee - Search Quality,Understand that the majority of your team lead...,"""Frustrating. No room for growth in certain de..."
4,1538929149-980,Good commission structure\nUnlimited PTO\nHapp...,Metrics focused\nNo one can explain how quotas...,"Nov 7, 2017","<span class=""rating""><span class=""value-title""...",I have been working at Indeed full-time (More ...,Recommends,Current Employee - Account Executive,Stop focusing on dials,"""Good Millennial Company"""


Basic preprocessing

In [53]:
#remove \n, \r, turn lowercase, then combine each word together for each review in one specific column, return that column
def word(dataframe, column_name):
    newdf=dataframe.copy()
    for i in range(len(newdf)):
        if type(newdf.iloc[i][column_name]) is str:
            newdf.iloc[i][column_name]=newdf.iloc[i][column_name].lower()
            newdf.iloc[i][column_name]=' '.join([s for s in newdf.iloc[i][column_name].split()])
    return newdf[column_name]

In [41]:
#remove punctuation, then combine each word together for each review in one specific column, return that column
def punc(dataframe, column_name):
    newdf=dataframe.copy()
    for i in range(len(newdf)):
        if type(newdf.iloc[i][column_name]) is str:
            newdf.iloc[i][column_name]= ''.join([c for c in newdf.iloc[i][column_name] if c not in string.punctuation])
    return newdf[column_name]

In [42]:
#remove stopwords in each review for one specific column, return that column
def stopword(dataframe, column_name):
    stop_words = stopwords.words('english')
    newdf=dataframe.copy()
    for i in range(len(newdf)):
        if type(newdf.iloc[i][column_name]) is str:
            newdf.iloc[i][column_name]= ' '.join([s for s in newdf.iloc[i][column_name].split() if s not in stop_words])
    return newdf[column_name]

In [84]:
#combine them together to save running time
def combine(dataframe, column_name):
    newdf=dataframe.copy()
    stop_words = stopwords.words('english')
    for i in range(len(newdf)):
        if type(newdf.iloc[i][column_name]) is str:
            newdf.iloc[i][column_name]=newdf.iloc[i][column_name].lower()
            newdf.iloc[i][column_name]=' '.join([s for s in newdf.iloc[i][column_name].split()])
            newdf.iloc[i][column_name]= ''.join([c for c in newdf.iloc[i][column_name] if c not in string.punctuation])
            newdf.iloc[i][column_name]= ' '.join([s for s in newdf.iloc[i][column_name].split() if s not in stop_words])
    return newdf[column_name]

Extra functions

In [44]:
#deem each word as appearing only once in each review in one specific column, return that column (must run the function 'word' to remove \n \r first)
def no_dup(dataframe,column_name):
    newdf=dataframe.copy()
    for i in range(len(newdf)):
        if type(newdf.iloc[i][column_name]) is str:
            newdf.iloc[i][column_name]= ' '.join(str(s) for s in set((newdf.iloc[i][column_name].split())))
    return newdf[column_name]

In [45]:
#combine the reviews for one specific column into one long string (must run the function 'word' to remove \n \r first)
def to_one_string(dataframe,column_name):
    string=''
    for text in dataframe[column_name]:
        if type(text) is str:
            strs=' '.join(str(s) for s in text.split())
            string=string+strs
    return string

In [46]:
#count word frequency in one string
def freq(string):
    return pd.Series(string.split()).value_counts().sort_values(ascending=False)[:25]

In [47]:
#ngrams count in one string (incase we need)
#note that we might have to keep punctuations in this case because one ngram is not meaningful with a punctuation inside.
def ngram(string, num_gram):
    token=word_tokenize(string)
    grams=ngrams(token,num_gram)
    return Counter(grams)

In [68]:
ngram(df.iloc[0]['pros'],3)

Counter({('-', 'Big', 'Company'): 1,
         ('-', 'Big', 'social'): 1,
         ('-', 'Good', 'car'): 1,
         ('-', 'Interesting', 'activities'): 1,
         ('.', '-', 'Big'): 1,
         ('.', '-', 'Good'): 1,
         ('.', '-', 'Interesting'): 1,
         ('Big', 'Company', 'offering'): 1,
         ('Big', 'social', 'hub'): 1,
         ('Company', 'offering', 'job'): 1,
         ('Good', 'car', 'loaning'): 1,
         ('Interesting', 'activities', 'and'): 1,
         ('activities', 'and', 'oportunities'): 1,
         ('and', 'oportunities', '.'): 1,
         ('and', 'selling', 'deals'): 1,
         ('car', 'loaning', 'and'): 1,
         ('connections', '.', '-'): 1,
         ('for', 'connections', '.'): 1,
         ('hub', 'for', 'connections'): 1,
         ('job', 'stability', '.'): 1,
         ('loaning', 'and', 'selling'): 1,
         ('offering', 'job', 'stability'): 1,
         ('oportunities', '.', '-'): 1,
         ('selling', 'deals', '.'): 1,
         ('social', 'hub

In [85]:
new_df=df.copy()
new_df['time']=pd.to_datetime(new_df['time'])

In [86]:
new_df['pros']=combine(df,'pros')
new_df['cons']=combine(df,'cons')
new_df['advice']=combine(df,'advice')
new_df['title']=combine(df,'title')
new_df.head()

,web-scraper-order,pros,cons,time,rating,tenure,recommend,jobtitle,advice,title
0,1538929161-1013,1 simple product mean huge compliment capitali...,experience indeed truly tale two managers firs...,2017-08-07,"<span class=""rating""><span class=""value-title""...",I worked at Indeed full-time,NaN,Former Employee - Anonymous Employee,1 act globally dont talk time indeed could bes...,tale two managers
1,1538929049-740,great company really nice people feel like gre...,none whatsoever company great best work experi...,2016-07-28,"<span class=""rating""><span class=""value-title""...",I have been working at Indeed (More than a year),Recommends,Current Employee - Social Media Strategist/Mar...,dont,account executive
2,1538929155-995,career development programs bonuses based perf...,becoming big corporate indeed losing unique ch...,2017-09-14,"<span class=""rating""><span class=""value-title""...",I worked at Indeed full-time,Recommends,Former Employee - Anonymous Employee,improve communication,client services
3,1538929026-683,unlimited pto great team free food,favoritism lack professionalism knowledge uppe...,2017-03-01,"<span class=""rating""><span class=""value-title""...",I have been working at Indeed full-time (More ...,Doesn't Recommend,Current Employee - Search Quality,understand majority team leads new management ...,frustrating room growth certain departments va...
4,1538929149-980,good commission structure unlimited pto happy ...,metrics focused one explain quotas generated,2017-11-07,"<span class=""rating""><span class=""value-title""...",I have been working at Indeed full-time (More ...,Recommends,Current Employee - Account Executive,stop focusing dials,good millennial company


In [87]:
new_df.to_csv('processed_indeed.csv')

In [14]:
freq(to_one_string(new_df,'pros'))

work             683
good             435
great            414
people           331
opportunities    242
company          229
benefits         204
accenture        170
career           158
place            148
projects         132
get              128
training         120
environment      119
culture          118
different        118
lot              118
balance          112
working          104
learn            102
lots             100
opportunity       96
experience        95
time              94
pay               93
dtype: int64